In [2]:
import os
import subprocess
# Check if the repository has already been cloned
repo_name = "Digital-Egypt-Pioneers-Initiative-DEPI-"
repo_url = "https://github.com/youssef-223/Digital-Egypt-Pioneers-Initiative-DEPI-.git"

In [2]:
if os.path.exists(repo_name):
        print(f"Repository '{repo_name}' already exists. Pulling latest changes.")
        subprocess.run(['git', '-C', "Digital-Egypt-Pioneers-Initiative-DEPI-", 'pull'], check=True)
else:
        print(f"Cloning repository '{repo_name}'.")
        subprocess.run(["git", "clone", repo_url])


Repository 'Digital-Egypt-Pioneers-Initiative-DEPI-' already exists. Pulling latest changes.


In [3]:
# Change directory to the desired folder
target_dir = os.path.join(repo_name, 'Microsoft_Malware_Kaagle')

if os.path.exists(target_dir):
        os.chdir(target_dir)
        print(f"Changed directory to {target_dir}.")
else:
        print(f"Target directory '{target_dir}' not found.")

Changed directory to Digital-Egypt-Pioneers-Initiative-DEPI-/Microsoft_Malware_Kaagle.


In [4]:
from google.colab import files


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display


from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif, VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.preprocessing import LabelEncoder

from imblearn.over_sampling import SMOTE

from utils import *


In [6]:
! pip install -q kaggle
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c microsoft-malware-prediction

! unzip "microsoft-malware-prediction.zip"

ERROR: Operation cancelled by user


KeyboardInterrupt: 

# Step1: EDA

In [26]:
train_data = load_random_sample('train.csv', sample_size=10000, random_seed =42)


In [7]:
train_data.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000d2c33cde94a58c7d265850b87a67,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1209.0,0,7.0,0,NaN,45157.0,...,45538.0,0,NaN,0.0,0,0,0.0,0.0,15.0,0
1,00098f9e3bc24337b4605a9b041e3636,win8defender,1.1.15200.1,4.18.1807.18075,1.275.423.0,0,7.0,0,NaN,53447.0,...,10814.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000c02b627970574ed9fcf4f09d6e1f1,win8defender,1.1.15300.5,4.18.1807.18075,1.275.1140.0,0,7.0,0,NaN,68585.0,...,46596.0,1,NaN,0.0,0,0,0.0,1.0,7.0,1
3,000edc601b0d7e6a56e51811e3cd024e,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1187.0,0,7.0,0,NaN,53447.0,...,33095.0,0,NaN,0.0,0,0,0.0,0.0,10.0,1
4,000fee2a9961f2974d49164e3e57d713,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1490.0,0,7.0,0,NaN,53447.0,...,20107.0,0,0.0,0.0,0,0,0.0,0.0,7.0,1


In [13]:
num_features = train_data.shape[1]
print(f"Number of features: {num_features}")
training_examples = train_data.shape[0]
print(f"Number of training examples: {training_examples}")

Number of features: 83
Number of training examples: 10000


In [10]:
train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
IsBeta,10000.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00
RtpStateBitfield,9965.0,6.858705e+00,9.832528e-01,0.0,7.0,7.0,7.000000e+00,8.000000e+00
IsSxsPassiveMode,10000.0,1.570000e-02,1.243184e-01,0.0,0.0,0.0,0.000000e+00,1.000000e+00
DefaultBrowsersIdentifier,497.0,1.691235e+03,9.950194e+02,24.0,878.0,1632.0,2.388000e+03,3.195000e+03
AVProductStatesIdentifier,9959.0,4.759631e+04,1.427700e+04,901.0,47490.0,53447.0,5.344700e+04,7.026200e+04
AVProductsInstalled,9959.0,1.338086e+00,5.286209e-01,1.0,1.0,1.0,2.000000e+00,4.000000e+00
AVProductsEnabled,9959.0,1.022392e+00,1.706546e-01,0.0,1.0,1.0,1.000000e+00,4.000000e+00
HasTpm,10000.0,9.886000e-01,1.061658e-01,0.0,1.0,1.0,1.000000e+00,1.000000e+00
CountryIdentifier,10000.0,1.075180e+02,6.333590e+01,1.0,50.0,97.0,1.630000e+02,2.220000e+02
CityIdentifier,9614.0,8.132409e+04,4.911542e+04,41.0,37026.0,82373.0,1.247360e+05,1.679070e+05


In [11]:
def get_numerical_columns_with_few_unique_values_and_object_columns(df, max_unique=10, show_unique_values=False):
    # Initialize sets to hold column names
    numerical_with_few_unique = set()
    all_object_columns = set()

    # Iterate over columns to check unique values
    for column in df.columns:
        # Check for numerical columns with fewer than max_unique unique values
        if pd.api.types.is_numeric_dtype(df[column]):
            num_unique = len(df[column].unique())
            if num_unique < max_unique:
                numerical_with_few_unique.add(column)

        # Check for object type columns
        elif pd.api.types.is_object_dtype(df[column]):
            all_object_columns.add(column)

    # Combine both sets into one list
    combined_columns = list(numerical_with_few_unique.union(all_object_columns))

    # Print the results for verification
    if show_unique_values:
        print("Numerical columns with fewer than", max_unique, "unique values:")
        for column in numerical_with_few_unique:
            unique_values = df[column].unique()
            print(f"Column: '{column}'")
            print(f"Number of unique values: {len(unique_values)}")
            print(f"Unique values: {unique_values}")
            print("-" * 100, "\n")

        print("All categorical columns with dtype 'object':")
        for column in all_object_columns:
            unique_values = df[column].unique()
            print(f"Column: '{column}'")
            print(f"Number of unique values: {len(unique_values)}")
            print(f"Unique values: {unique_values}")
            print("-" * 100, "\n")

    # Return the combined list of columns
    return combined_columns


In [27]:
cat_cols = get_numerical_columns_with_few_unique_values_and_object_columns(train_data,10,show_unique_values = False)

# Get the remaining columns that are not in cat_cols
num_cols = list(set(train_data.columns) - set(cat_cols))

# Print results
print(f"Categorical columns with few unique values: {cat_cols}")
print(f"Number of categorical features: {len(cat_cols)}/{num_features}")
print(f"Numerical columns (rest of the columns): {num_cols}")
print(f"Number of numerical features: {len(num_cols)}/{num_features}")

Categorical columns with few unique values: ['Firewall', 'MachineIdentifier', 'OsBuildLab', 'AppVersion', 'AutoSampleOptIn', 'Census_IsAlwaysOnAlwaysConnectedCapable', 'HasTpm', 'SkuEdition', 'Census_IsWIMBootEnabled', 'OsSuite', 'Wdft_IsGamer', 'Census_InternalBatteryType', 'IsSxsPassiveMode', 'Census_IsVirtualDevice', 'Census_GenuineStateName', 'Census_PrimaryDiskTypeName', 'Census_IsPortableOperatingSystem', 'RtpStateBitfield', 'SmartScreen', 'IsProtected', 'Platform', 'Census_IsFlightsDisabled', 'Processor', 'AVProductsEnabled', 'UacLuaenable', 'Census_OSEdition', 'Census_OSWUAutoUpdateOptionsName', 'Census_DeviceFamily', 'Census_OSSkuName', 'PuaMode', 'ProductName', 'Census_ProcessorManufacturerIdentifier', 'Census_OSArchitecture', 'IsBeta', 'HasDetections', 'Census_IsFlightingInternal', 'Census_ActivationChannel', 'Census_IsSecureBootEnabled', 'OsPlatformSubRelease', 'Census_OSBranch', 'Census_IsPenCapable', 'Census_MDC2FormFactor', 'EngineVersion', 'Census_ProcessorClass', 'Cens

In [31]:

# Create the DataFrame with missing values summary
missing_values_summary = pd.DataFrame({
    'Total No. of Missing Values': train_data.isnull().sum(),
    'Percentage of Missing Values': (train_data.isnull().sum() / len(train_data)) * 100
})

# Display the DataFrame
display(missing_values_summary)

,Total No. of Missing Values,Percentage of Missing Values
MachineIdentifier,0,0.00
ProductName,0,0.00
EngineVersion,0,0.00
AppVersion,0,0.00
AvSigVersion,0,0.00
...,...,...
Census_IsPenCapable,0,0.00
Census_IsAlwaysOnAlwaysConnectedCapable,94,0.94
Wdft_IsGamer,377,3.77
Wdft_RegionIdentifier,377,3.77


In [16]:
def downcast_numerical(df, num_cols):
    for col in num_cols:
        if df[col].dtype in ['float64', 'int64']:  # Check if column is numerical
            df[col] = pd.to_numeric(df[col], downcast='integer')
    return df


def encode_categorical(df, cat_cols):
    label_encoders = {}

    for col in cat_cols:
        if col in df.columns:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
            label_encoders[col] = le

    return df

In [19]:
# Print memory usage before optimization
print(">>Memory usage before optimization:")
memory_before = train_data.memory_usage(deep=True)
print(memory_before)

# Perform optimizations
optimized_data = downcast_numerical(train_data.copy(), num_cols)
optimized_data = encode_categorical(optimized_data, cat_cols)

# Print memory usage after optimization
print("-" * 100, "\n")
print(">>Memory usage after optimization:")
memory_after = optimized_data.memory_usage(deep=True)
print(memory_after)

# Calculate and print memory utilization changes
memory_before_total = memory_before.sum()
memory_after_total = memory_after.sum()

print("-" * 100, "\n")
print(f"Total memory usage before optimization: {memory_before_total / (1024 ** 2):.2f} MB")
print(f"Total memory usage after optimization: {memory_after_total / (1024 ** 2):.2f} MB")
print(f"Memory saved: {(memory_before_total - memory_after_total) / (1024 ** 2):.2f} MB")

>>Memory usage before optimization:
Index                                        128
MachineIdentifier                          80000
ProductName                                80000
EngineVersion                              80000
AppVersion                                 80000
                                           ...  
Census_IsPenCapable                        80000
Census_IsAlwaysOnAlwaysConnectedCapable    80000
Wdft_IsGamer                               80000
Wdft_RegionIdentifier                      80000
HasDetections                              80000
Length: 84, dtype: int64
---------------------------------------------------------------------------------------------------- 

>>Memory usage after optimization:
Index                                        128
MachineIdentifier                          80000
ProductName                                80000
EngineVersion                              80000
AppVersion                                 80000
                 

In [32]:

# Create the DataFrame with missing values summary
missing_values_summary = pd.DataFrame({
    'Total No. of Missing Values': train_data.isnull().sum(),
    'Percentage of Missing Values': (train_data.isnull().sum() / len(train_data)) * 100
})

# Display the DataFrame
display(missing_values_summary)

,Total No. of Missing Values,Percentage of Missing Values
MachineIdentifier,0,0.00
ProductName,0,0.00
EngineVersion,0,0.00
AppVersion,0,0.00
AvSigVersion,0,0.00
...,...,...
Census_IsPenCapable,0,0.00
Census_IsAlwaysOnAlwaysConnectedCapable,94,0.94
Wdft_IsGamer,377,3.77
Wdft_RegionIdentifier,377,3.77


In [33]:
train_data_cleaned = clean_and_impute_data(train_data, cat_cols, num_cols)


Dropped features with more than 50% missing values:
['DefaultBrowsersIdentifier', 'PuaMode', 'Census_ProcessorClass', 'Census_InternalBatteryType', 'Census_IsFlightingInternal', 'Census_ThresholdOptIn', 'Census_IsWIMBootEnabled']
New DataFrame shape: (10000, 76)
Old DataFrame shape: (10000, 83)


In [ ]:
# Usage
plot_numerical_features(train_data)


In [ ]:
# Count the values in the target column 'Wdft_IsGamer'
count = train_data["Wdft_IsGamer"].value_counts()

# Create a figure for the plots
plt.figure(figsize=(15,6))

# Bar plot for the 'Wdft_IsGamer' distribution
plt.subplot(1,2,1)
ax = sns.countplot(y="Wdft_IsGamer", data=train_data, hue="Wdft_IsGamer", palette="Set2")
ax.bar_label(ax.containers[0], fontweight="black", size=15)
plt.title("Wdft_IsGamer Distribution", fontweight="black", size=20, pad=20)

# Pie chart for the 'Wdft_IsGamer' distribution
plt.subplot(1,2,2)
plt.pie(count.values, labels=count.index, autopct="%1.1f%%", colors=sns.set_palette("Set2"),
        textprops={"fontweight": "black"}, explode=[0, 0.1])
plt.title("Wdft_IsGamer Distribution", fontweight="black", size=20, pad=20)

# Show the plot
plt.show()


In [ ]:
countplot(train_data, 'Census_IsAlwaysOnAlwaysConnectedCapable')